# Hate Speech Detection Model
Fine-tuning RoBERTa on the combined hate speech dataset for 3-way classification.

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score, classification_report

## Load and Prepare Data

In [ ]:
# Load the cleaned and encoded combined dataset
df = pd.read_csv("combined_hate_speech_dataset.csv")
df['label_encoded'] = pd.factorize(df['label'])[0]  # Ensure encoding if not saved before

# Train/Val/Test split
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, stratify=df['label_encoded'], test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, stratify=temp_df['label_encoded'], test_size=0.5, random_state=42)

## Convert to Hugging Face Datasets

In [ ]:
train_dataset = Dataset.from_pandas(train_df[['text', 'label_encoded']])
val_dataset = Dataset.from_pandas(val_df[['text', 'label_encoded']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label_encoded']])

## Load RoBERTa Model and Tokenizer

In [ ]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

## Tokenization Function

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

## Define Metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_macro": f1_score(labels, predictions, average='macro')
    }

## Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Train the Model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

## Evaluate on Test Set

In [ ]:
predictions = trainer.predict(test_dataset)
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)
print(classification_report(y_true, y_pred))